To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B-Instruct-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Glaive Function Calling dataset from [madroid](https://huggingface.co/datasets/madroid/glaive-function-calling-openai), which is a version of the original [Glaive Function Calling v2](https://huggingface.co/datasets/glaiveai/glaive-function-calling-v2) pre-processed to facilitate integration. You can replace this code section with your own data prep.

**[NOTE]** Each model has its own Tool Calling template. For `qwen-2.5` we'll use the [official template](https://qwen.readthedocs.io/en/latest/framework/function_call.html#hugging-face-transformers). If you want to use another model and/or template, you'll need to write your own data prep. See [this notebook](https://colab.research.google.com/drive/1-1FbzLnx1DWRa8ysx5KUlhvRtaToCbvV?usp=sharing) for a demo with `llama-3.1-8B`.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
#@title Process dataset util
import json
import random
from datasets import load_dataset
import ast


# This is our transformation function that creates the new chat format.
def get_formatted_sample(item):
    # Parse the JSON string from the dataset sample
    sample = item
    tools = sample.get("tools", [])

    # new_history will store the final sequence of messages.
    new_history = []
    # pending_assistant is used to merge consecutive assistant messages.
    pending_assistant = None
    # Mapping from tool call id to function name so we can later label the tool response.
    mapping_tool_id_to_function_name = {}

    # Process each message in the original sample.
    for msg in sample["messages"]:
        role = msg.get("role")

        if role == "system":
            # Flush any pending assistant message
            if pending_assistant is not None:
                new_history.append(pending_assistant)
                pending_assistant = None
            # Append system message as-is.
            new_history.append(msg)

        elif role == "user":
            # Flush any pending assistant message before adding a new user message.
            if pending_assistant is not None:
                new_history.append(pending_assistant)
                pending_assistant = None
            new_history.append(msg)

        elif role == "assistant":
            # If we haven't started merging an assistant message yet, start one.
            if pending_assistant is None:
                pending_assistant = {"role": "assistant", "content": "", "tool_calls": []}

            # Merge textual content if present.
            if "content" in msg and msg["content"]:
                if pending_assistant["content"]:
                    # Append on a new line if already exists.
                    pending_assistant["content"] += "\n" + msg["content"]
                else:
                    pending_assistant["content"] = msg["content"]

            # Process any tool_calls: remove the "id" field and record a mapping.
            if "tool_calls" in msg:
                for tc in msg["tool_calls"]:
                    # Map the id to function name if present.
                    if "id" in tc:
                        mapping_tool_id_to_function_name[tc["id"]] = tc["function"]["name"]
                    function_name = tc["function"]["name"]
                    arguments = tc["function"]["arguments"]
                    if isinstance(arguments, str):
                        arguments = ast.literal_eval(arguments)
                    pending_assistant["tool_calls"].append({
                        "name": function_name,
                        "arguments": arguments
                    })

        elif role == "tool":
            # For tool responses, we expect a tool_call_id that maps back to a tool call.
            tool_call_id = msg.get("tool_call_id")
            function_name = mapping_tool_id_to_function_name.get(tool_call_id, "")
            # Create a tool response message in the chat format (role 'user' with the "name" set to the function name).
            tool_response = {
                "role": "user",
                "name": function_name,
                "content": msg.get("content", "")
            }
            # Flush any pending assistant message before appending the tool response.
            if pending_assistant is not None:
                new_history.append(pending_assistant)
                pending_assistant = None
            new_history.append(tool_response)

        else:
            # For any unknown roles, flush and then append as-is.
            if pending_assistant is not None:
                new_history.append(pending_assistant)
                pending_assistant = None
            new_history.append(msg)

    # Flush any remaining pending assistant message.
    if pending_assistant is not None:
        new_history.append(pending_assistant)

    # Now apply the chat template to the reconstructed history.
    context = tokenizer.apply_chat_template(
        new_history,
        tools=tools,
        tokenize=False,
        add_generation_prompt=False,
    )
    return context

In [ ]:
#@title Process dataset

def formatting_prompts_func(examples):
    texts = []
    json_strs = examples["json"]
    for sample in json_strs:
        text = get_formatted_sample(json.loads(sample))
        texts.append(text)
    return { "text" : texts, }
pass

# from datasets import load_dataset
dataset = load_dataset("madroid/glaive-function-calling-openai", split = "train[:1024]")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/7.55k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/103M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112754 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/967 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0]["text"])

<|im_start|>system
You are a helpful assistant with access to the following functions. Use them if required

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "track_calories", "description": "Track daily calorie intake", "parameters": {"type": "object", "properties": {"meal": {"type": "string", "description": "The meal for which calories are being tracked"}, "calories": {"type": "number", "description": "The number of calories consumed"}, "date": {"type": "string", "format": "date", "description": "The date for which calories are being tracked"}}, "required": ["meal", "calories", "date"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Hi, I had 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/1024 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1024 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1024 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1024 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.525 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,024 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,1.631300
2,1.851800
3,1.406800
4,1.483500
5,1.298500
6,0.871300
7,1.042400
8,0.976100
9,0.873000
10,0.906900


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

259.5915 seconds used for training.
4.33 minutes used for training.
Peak reserved memory = 3.266 GB.
Peak reserved memory for training = 1.741 GB.
Peak reserved memory % of max memory = 22.156 %.
Peak reserved memory for training % of max memory = 11.811 %.


<a name="Inference"></a>
### Inference
Let's run the model!


**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
print(dataset[0]["text"])

<|im_start|>system
You are a helpful assistant with access to the following functions. Use them if required

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "track_calories", "description": "Track daily calorie intake", "parameters": {"type": "object", "properties": {"meal": {"type": "string", "description": "The meal for which calories are being tracked"}, "calories": {"type": "number", "description": "The number of calories consumed"}, "date": {"type": "string", "format": "date", "description": "The date for which calories are being tracked"}}, "required": ["meal", "calories", "date"]}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Hi, I had 

In [ ]:
#@title Tool using inference util
import re


# https://qwen.readthedocs.io/en/latest/framework/function_call.html#id3
def try_parse_tool_calls(content: str):
    """Try parse the tool calls."""
    tool_calls = []
    offset = 0
    for i, m in enumerate(re.finditer(r"<tool_call>\n(.+)?\n</tool_call>", content)):
        if i == 0:
            offset = m.start()
        try:
            func = json.loads(m.group(1))
            tool_calls.append({"type": "function", "function": func})
            if isinstance(func["arguments"], str):
                func["arguments"] = json.loads(func["arguments"])
        except json.JSONDecodeError as e:
            print(f"Failed to parse tool calls: the content is {m.group(1)} and {e}")
            pass
    if tool_calls:
        if offset > 0 and content[:offset].strip():
            c = content[:offset]
        else:
            c = ""
        return {"role": "assistant", "content": c, "tool_calls": tool_calls}
    return {"role": "assistant", "content": re.sub(r"<\|im_end\|>$", "", content)}


**Model determining which tool to call**

We feed the model with a user message and a list of tools. It responds with a tool call in the following format:
```xml
Looking into my database... One sec.
<tool_call>
{"name": "find_movie_details", "arguments": {"title": "Inception"}}
</tool_call><|im_end|>
```

**In order** to use qwen-2.5's native tool calling with `transformers`, we must define our functions with Python and pass them as a parameter during `apply_chat_template`.

**[NOTE]** to be correctly parsed a tool must have type annotations and a valid docstring.

In [ ]:
#@title Tool Calling inference
def find_movie_details(title: str):
  """Find details about a movie based on its title
  Args:
      title: The title of the movie

  Returns:
      dict: A dictionary containing the movie details
  """
  if title == "Inception":
      return {"title": "Inception", "director": "Christopher Nolan", "release_year": 2010, "genre": "Science Fiction", "rating": 8.8}
  elif title == "The Godfather":
      return {"title": "The Godfather", "director": "Francis Ford Coppola", "release_year": 1972, "genre": "Crime, Drama", "rating": 9.2}
  else:
      return {}


def play_music(genre: str, mood: str) -> None:
  """Play music based on user's preferences
  Args:
      genre: The genre of music to play
      mood: The mood of the music to play
  """
  pass


# easily accessible by name
function_name = {
    "find_movie_details": find_movie_details,
    "play_music": play_music,
}



FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [{
    "role": "user",
    "content": "Good morning! What do you know about Inception? Please provide me all your info on this movie."},
]


context = tokenizer.apply_chat_template(
    messages,
    tools=[find_movie_details, play_music],
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(
[
    context,
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

output_text = tokenizer.batch_decode(outputs)[0][len(context):]
print(output_text)

<tool_call>
{"name": "find_movie_details", "arguments": {"title": "Inception"}}
</tool_call><|im_end|>


**Result from calling the tool is passed back to the model and it generates the final response to the user**

Now we add the tool call from the previous generation and its result to the context, the model then generates the final response.

In [ ]:
messages.append(try_parse_tool_calls(output_text))

# https://qwen.readthedocs.io/en/latest/framework/function_call.html#id3
if tool_calls := messages[-1].get("tool_calls", None):
    for tool_call in tool_calls:
        if fn_call := tool_call.get("function"):
            fn_name: str = fn_call["name"]
            fn_args: dict = fn_call["arguments"]

            fn_res: str = json.dumps(function_name[fn_name](**fn_args))

            messages.append({
                "role": "tool",
                "name": fn_name,
                "content": fn_res,
            })

context = tokenizer.apply_chat_template(
    messages,
    tools=[find_movie_details, play_music],
    tokenize=False,
    add_generation_prompt=True,
)

print(context)
print("==="*10)

inputs = tokenizer(
[
    context,
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

output_text = tokenizer.batch_decode(outputs)[0][len(context):]

print(output_text)

Inception is a science fiction film directed by Christopher Nolan that was released in 2010. It has an average rating of 8.8.<|im_end|>


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [{
    "role": "user",
    "content": "Please i want to listen some blues. play immediately"},
]


context = tokenizer.apply_chat_template(
    messages,
    tools=[find_movie_details, play_music],
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(
[
    context,
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

output_text = tokenizer.batch_decode(outputs)[0][len(context):]
print(output_text)

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<tool_call>
{"name": "play_music", "arguments": {"genre": "blues", "mood": "relaxed"}}
</tool_call><|im_end|>
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "find_movie_details", "description": "Find details about a movie based on its title", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie"}}, "required": ["title"]}}}
{"type": "function", "function": {"name": "play_music", "description": "Play music based on user's preferences", "parameters": {"type": "object", "properties": {"genre": {"type": "string", "description": "The genre of music to play"}, "mood": {"type": "string", "description": "The mood of the music to play"}}, "required": ["genre", "mood"]}, "return": {"type": "null"}

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


messages = [{
    "role": "user",
    "content": "Please i want to listen some blues"},
]


context = tokenizer.apply_chat_template(
    messages,
    tools=[find_movie_details, play_music],
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(
[
    context,
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

output_text = tokenizer.batch_decode(outputs)[0][len(context):]
print(output_text)

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
